In [ ]:
!pip install langchain langchain-openai tiktoken slack_sdk langchain_community

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SLACK_API_TOKEN = userdata.get('SLACK_API_TOKEN')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

from slack_sdk import WebClient
# 슬랙 클라이언트 초기화
client = WebClient(token=SLACK_API_TOKEN)

def fetch_slack_thread(channel_id, thread_ts):
    try:
        response = client.conversations_replies(
            channel=channel_id,
            ts=thread_ts
        )
        return response['messages']
    except SlackApiError as e:
        print(f"Error fetching thread: {e.response['error']}")
        return None

def extract_relevant_info(messages):
    # 메시지 내용을 추출
    messages_text = [msg['text'] for msg in messages]
    return "\n".join(messages_text)

prompt_template = ChatPromptTemplate.from_template("""
당신은 AI 비서입니다. 당신에게 주어진 슬랙 스레드는 장애에 대해 논의하고 있습니다. 이 스레드를 바탕으로 장애 보고서를 작성하십시오.
장애 보고서 템플릿은 아래와 같아.
타임 라인 및 원인 분석

1. 타임라인
아래 기준으로 작성 후 우측 Details 시간 정보에도 함께 업데이트 해주세요. (직접 입력시 분 단위로 작성 가능합니다.)
내용 작성시 참고한 데이터의 링크도 함께 넣어주세요.
* 장애시작
    * 시간:
    * 내용:
* 장애탐지
    * 시간:
    * 확인방법:
    * 장애공유:
* 장애복구
    * 시간:
    * 원인:
    * 조치:

2. 장애원인 (Root Cause)

장애가 발생한 근본적인 원인을 작성해주세요.

설명

재발방지 대책 (poka-yoke)
 답변은 한글로 작성해 주세요.

스레드: {thread_text}

장애 보고서:
""")
# LCEL chaining
chain = LLMChain(
    llm=ChatOpenAI(model="gpt-4", openai_api_key=OPENAI_API_KEY),
    prompt=prompt_template,
    output_parser=StrOutputParser()
)

# 슬랙 스레드 가져오기
channel_id = "CQXDCMWAC"
thread_ts = "1721028636.946999"  # 스레드의 타임스탬프

messages = fetch_slack_thread(channel_id, thread_ts)
if messages:
    thread_text = extract_relevant_info(messages)

    # 텍스트 분할
    splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
    chunks = splitter.split_text(thread_text)

    # 각 조각을 처리하고 결합
    incident_reports = []
    for chunk in chunks:
        report = chain.run(chunk)
        incident_reports.append(report)

    final_report = "\n".join(incident_reports)
    print("Incident Report:")
    print(final_report)
else:
    print("Failed to fetch the thread.")